In [45]:
import pandas as pd
import torch
import torch.nn as nn

In [46]:
#df = pd.read_pickle("papila_embedding/test_df.pickle")

In [47]:
df.head(1)

,Diagnosis,Folder,Sex,Patient Age,Weight,Clinical characteristics,Surgery,Follow-up,PCR,Path,Age,Age_multi,Age_binary,binary_label
0,COVID-19,P001,M,039Y,80.0,"Dyspnea, Cough, Fever",NaN,NaN,NaN,COVID-19 Cases 3D/P001.npy,39,1,0,1


In [80]:
from torch.nn.modules import activation
from torch.nn.modules.linear import Linear
from torch.nn.modules.activation import Sigmoid
from collections import Counter

def get_df_info(dataframe, fairness=True):
    embedding = dataframe["embedding"][0].flatten()

    if fairness is True:
        age = -1
        gender = -1

        age = torch.tensor([age])
        gender = torch.tensor([gender])

    else:
        age = torch.tensor([dataframe["Age_binary"][0]])
        gender = torch.tensor([dataframe["Sex"][0]])

    sample_input = embedding#torch.cat((age, gender, embedding), dim=0)
    output_len  = len(set(dataframe['binaryLabel']))

    return len(sample_input), output_len

In [49]:
!pip install torchio

In [50]:
import numpy as np
import torch
import torchvision.transforms as transforms
import pandas as pd
import pickle
import random
import torchio as tio
# from utils.spatial_transforms import ToTensor

from torchvision.transforms._transforms_video import (
    NormalizeVideo,
)

from torch.utils.data import WeightedRandomSampler
import json


class BaseDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, sens_name, sens_classes):
        super(BaseDataset, self).__init__()

        self.dataframe = dataframe
        self.dataset_size = self.dataframe.shape[0]
        self.sens_name = sens_name
        self.sens_classes = sens_classes

        self.A = None
        self.Y = None
        self.AY_proportion = None

    def get_AY_proportions(self):
        if self.AY_proportion:
            return self.AY_proportion

        A_num_class = 2
        Y_num_class = 2
        A_label = self.A
        Y_label = self.Y

        A = self.A.tolist()
        Y = self.Y.tolist()
        ttl = len(A)

        len_A0Y0 = len([ay for ay in zip(A, Y) if ay == (0, 0)])
        len_A0Y1 = len([ay for ay in zip(A, Y) if ay == (0, 1)])
        len_A1Y0 = len([ay for ay in zip(A, Y) if ay == (1, 0)])
        len_A1Y1 = len([ay for ay in zip(A, Y) if ay == (1, 1)])

        assert (
                       len_A0Y0 + len_A0Y1 + len_A1Y0 + len_A1Y1
               ) == ttl, "Problem computing train set AY proportion."
        A0Y0 = len_A0Y0 / ttl
        A0Y1 = len_A0Y1 / ttl
        A1Y0 = len_A1Y0 / ttl
        A1Y1 = len_A1Y1 / ttl

        self.AY_proportion = [[A0Y0, A0Y1], [A1Y0, A1Y1]]

        return self.AY_proportion

    def get_A_proportions(self):
        AY = self.get_AY_proportions()
        ret = [AY[0][0] + AY[0][1], AY[1][0] + AY[1][1]]
        np.testing.assert_almost_equal(np.sum(ret), 1.0)
        return ret

    def get_Y_proportions(self):
        AY = self.get_AY_proportions()
        ret = [AY[0][0] + AY[1][0], AY[0][1] + AY[1][1]]
        np.testing.assert_almost_equal(np.sum(ret), 1.0)
        return ret

    def set_A(self, sens_name):
        if sens_name == 'Sex':
            A = np.asarray(self.dataframe['Sex'].values != 'M').astype('float')
        elif sens_name == 'Age':
            A = np.asarray(self.dataframe['Age_binary'].values.astype('int') == 1).astype('float')
        elif sens_name == 'Race':
            A = np.asarray(self.dataframe['Race'].values == 'White').astype('float')
        elif self.sens_name == 'skin_type':
            A = np.asarray(self.dataframe['skin_binary'].values != 0).astype('float')
        elif self.sens_name == 'Insurance':
            self.A = np.asarray(self.dataframe['Insurance_binary'].values != 0).astype('float')
        else:
            raise ValueError("Does not contain {}".format(self.sens_name))
        return A

    def get_weights(self, resample_which):
        sens_attr, group_num = self.group_counts(resample_which)
        group_weights = [1/x.item() for x in group_num]
        sample_weights = [group_weights[int(i)] for i in sens_attr]
        return sample_weights

    def group_counts(self, resample_which = 'group'):
        if resample_which == 'group' or resample_which == 'balanced':
            if self.sens_name == 'Sex':
                mapping = {'M': 0, 'F': 1}
                groups = self.dataframe['Sex'].values
                group_array = [*map(mapping.get, groups)]

            elif self.sens_name == 'Age':
                if self.sens_classes == 2:
                    groups = self.dataframe['Age_binary'].values
                elif self.sens_classes == 5:
                    groups = self.dataframe['Age_multi'].values
                elif self.sens_classes == 4:
                    groups = self.dataframe['Age_multi4'].values.astype('int')
                group_array = groups.tolist()

            elif self.sens_name == 'Race':
                mapping = {'White': 0, 'non-White': 1}
                groups = self.dataframe['Race'].values
                group_array = [*map(mapping.get, groups)]
            elif self.sens_name == 'skin_type':
                if self.sens_classes == 2:
                    groups = self.dataframe['skin_binary'].values
                elif self.sens_classes == 6:
                    groups = self.dataframe['skin_type'].values
                group_array = groups.tolist()
            elif self.sens_name == 'Insurance':
                if self.sens_classes == 2:
                    groups = self.dataframe['Insurance_binary'].values
                elif self.sens_classes == 5:
                    groups = self.dataframe['Insurance'].values
                group_array = groups.tolist()
            else:
                raise ValueError("sensitive attribute does not defined in BaseDataset")

            if resample_which == 'balanced':
                #get class
                labels = self.Y.tolist()
                num_labels = len(set(labels))
                num_groups = len(set(group_array))

                group_array = (np.asarray(group_array) * num_labels + np.asarray(labels)).tolist()

        elif resample_which == 'class':
            group_array = self.Y.tolist()
            num_labels = len(set(group_array))

        self._group_array = torch.LongTensor(group_array)
        if resample_which == 'group':
            self._group_counts = (torch.arange(self.sens_classes).unsqueeze(1)==self._group_array).sum(1).float()
        elif resample_which == 'balanced':
            self._group_counts = (torch.arange(num_labels * num_groups).unsqueeze(1)==self._group_array).sum(1).float()
        elif resample_which == 'class':
            self._group_counts = (torch.arange(num_labels).unsqueeze(1)==self._group_array).sum(1).float()
        return group_array, self._group_counts

    def __len__(self):
        return self.dataset_size

    def get_labels(self):
        # for sensitive attribute imbalance
        if self.sens_classes == 2:
            return self.A
        elif self.sens_classes == 5:
            return self.dataframe['Age_multi'].values.tolist()
        elif self.sens_classes == 4:
            return self.dataframe['Age_multi4'].values.tolist()

    def get_sensitive(self, sens_name, sens_classes, item):
        if sens_name == 'Sex':
            if item['Sex'] == 'M':
                sensitive = 0
            else:
                sensitive = 1
        elif sens_name == 'Age':
            if sens_classes == 2:
                sensitive = int(item['Age_binary'])
            elif sens_classes == 5:
                sensitive = int(item['Age_multi'])
            elif sens_classes == 4:
                sensitive = int(item['Age_multi4'])
        elif sens_name == 'Race':
            if item['Race'] == 'White':
                sensitive = 0
            else:
                sensitive = 1
        elif sens_name == 'skin_type':
            if sens_classes == 2:
                sensitive = int(item['skin_binary'])
            else:
                sensitive = int(item['skin_type'])
        elif self.sens_name == 'Insurance':
            if self.sens_classes == 2:
                sensitive = int(item['Insurance_binary'])
            elif self.sens_classes == 5:
                sensitive = int(item['Insurance'])
        else:
            raise ValueError('Please check the sensitive attributes.')
        return sensitive


class PAPILA(BaseDataset):
    def __init__(self, dataframe, sens_name, sens_classes):
        super(PAPILA, self).__init__(dataframe, sens_name, sens_classes)

        self.A = self.set_A(sens_name)
        self.Y = (np.asarray(self.dataframe['binaryLabel'].values) > 0).astype('float')
        self.AY_proportion = None

    def __getitem__(self, idx):
        item = self.dataframe.iloc[idx]
        idd = item['ID']
        
        embedding = item['embedding']

        label = torch.FloatTensor([item['binaryLabel']])

        sensitive = self.get_sensitive(self.sens_name, self.sens_classes, item)
        age = item['Age_binary']
        
        return idx, embedding, label, sensitive, age

In [51]:
f = open("papila_embedding/datasets.json")
data_setting = json.load(f)['PAPILA']
val_test_classes = val_test_classes = data_setting['sens_classes']

train_meta = pd.read_pickle('papila_embedding/train_df.pickle')
val_meta = pd.read_pickle('papila_embedding/val_df.pickle')
test_meta = pd.read_pickle('papila_embedding/test_df.pickle')

In [52]:
train_meta.shape,val_meta.shape,test_meta.shape

((195, 20), (24, 20), (25, 20))

In [62]:
train_meta.head(1)

,Unnamed: 0,ID,Age,Sex,Diagnosis,dioptre_1,dioptre_2,astigmatism,Phakic/Pseudophakic,Pneumatic,Perkins,Pachymetry,Axial_Length,VF_MD,PathOS,PathOD,Age_multi,Age_binary,binaryLabel,embedding
0,0,2,47,0,2,0.75,-1.75,90.0,0.0,21.0,NaN,586.0,23.64,-0.07,../../papila/Fundusimages/RET002OS.jpg,../../papila/Fundusimages/RET002OD.jpg,2,0,1,"[[[tensor(0.1418), tensor(0.0422), tensor(-0.0..."


In [67]:
val_meta.Age_binary.value_counts()

1    14
0    10
Name: Age_binary, dtype: int64

In [72]:
train_meta.loc[train_meta.Age_binary==0].binaryLabel.value_counts()

0    63
1    21
Name: binaryLabel, dtype: int64

In [73]:
train_meta.loc[train_meta.Age_binary==1].binaryLabel.value_counts()

0    74
1    37
Name: binaryLabel, dtype: int64

In [74]:
val_meta.binaryLabel.value_counts()

0    16
1     8
Name: binaryLabel, dtype: int64

In [75]:
val_meta.loc[val_meta.Sex==1].binaryLabel.value_counts()

0    11
1     3
Name: binaryLabel, dtype: int64

In [76]:
3/(3+11)

0.21428571428571427

In [43]:
val_meta.loc[val_meta.Sex==0].binaryLabel.value_counts()

1    5
0    5
Name: binaryLabel, dtype: int64

In [44]:
5/(5+5)

0.5

In [77]:
train_data = PAPILA(train_meta, data_setting['sensitive_name'], data_setting['sens_classes'])
val_data = PAPILA(val_meta, data_setting['sensitive_name'], val_test_classes)
test_data = PAPILA(test_meta, data_setting['sensitive_name'], val_test_classes)

/var/folders/nl/2sq00v7j1jxdw2y7s4_913tw0000gn/T/ipykernel_18073/287149947.py:75: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  A = np.asarray(self.dataframe['Sex'].values != 'M').astype('float')


In [78]:
class FCNet(nn.Module):

    def __init__(self, input_dim, output_dim):
        super().__init__()

        self.fc1 = nn.Linear(input_dim, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 32)
        self.fc4 = nn.Linear(32, output_dim)

        self.sig = nn.Sigmoid()
        self.softmax = nn.Softmax()
        self.dropout = nn.Dropout()

  
    def forward(self, embedding, age, gender):
        embedding = embedding.flatten()
        age = torch.tensor([age])
        gender = torch.tensor([gender])

        inputs = torch.cat((age, gender, embedding), dim=0)
    
        x = self.fc1(inputs)
        x = self.sig(x)
        x = self.fc2(x)
        x = self.sig(x)
        x = self.fc3(x)
        x = self.sig(x)
        x = self.fc4(x)
        outputs = self.softmax(x)

        return outputs

In [79]:
get_df_info(df)

KeyError: 'embedding'

In [152]:
net = FCNet(get_df_info(df)[0], get_df_info(df)[1])

In [155]:
for (idx, emb, lab, sex, age) in train_data:
    print(lab)
    break

tensor([1.])


In [156]:
def one_hot_encoding(num_class, label):
    res = [0*i for i in range(num_class)]
    res[int(label)] = 1

    return torch.tensor(res, dtype=torch.float)

In [159]:
#fair=true
import os
#from os import setxattr
import torch.optim as optim

epochs = 20
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(net.parameters(), lr=0.001)

min_valid_loss = np.inf

for e in range(epochs):
    running_loss = 0.0  
    for (idx, emb, lab, sex, age) in train_data:
        optimizer.zero_grad()

        embedding = emb
        age = age
        gender = sex

        labels = one_hot_encoding(get_df_info(df)[1], lab)

        outputs = net(embedding, age, gender)
        # print(outputs, labels)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    # if i % 10 == 9:
    #   print(f'[{e + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
    #   running_loss = 0.0
  
    valid_loss = 0.0
    net.eval()
    for (idx, emb, lab, sex, age) in val_data:
        embedding = emb
        age = age
        gender = sex

        labels = one_hot_encoding(get_df_info(df)[1], lab)


        target = net(embedding, age, gender)
        loss = criterion(outputs, labels)
        valid_loss = loss.item() * len(val_data)

    print(f'Epoch {e+1} \t\t Training Loss: {running_loss / len(train_data)} \t\t Validation Loss: {valid_loss / len(val_data)}')

    if min_valid_loss > valid_loss:
        print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
        min_valid_loss = valid_loss
        # Saving State Dict
        torch.save(net.state_dict(), 'saved_model.pth')

/var/folders/nl/2sq00v7j1jxdw2y7s4_913tw0000gn/T/ipykernel_80289/2076255210.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  outputs = self.softmax(x)


Epoch 1 		 Training Loss: 0.6107048389239189 		 Validation Loss: 0.31327298283576965
Validation Loss Decreased(inf--->7.518552) 	 Saving The Model
Epoch 2 		 Training Loss: 0.6106988080036946 		 Validation Loss: 0.3132708966732025
Validation Loss Decreased(7.518552--->7.518502) 	 Saving The Model
Epoch 3 		 Training Loss: 0.6106982142497331 		 Validation Loss: 0.3132694959640503
Validation Loss Decreased(7.518502--->7.518468) 	 Saving The Model
Epoch 4 		 Training Loss: 0.6106980030353253 		 Validation Loss: 0.31326836347579956
Validation Loss Decreased(7.518468--->7.518441) 	 Saving The Model
Epoch 5 		 Training Loss: 0.6106978994149428 		 Validation Loss: 0.3132675886154175
Validation Loss Decreased(7.518441--->7.518422) 	 Saving The Model
Epoch 6 		 Training Loss: 0.6106978326271741 		 Validation Loss: 0.3132668733596802
Validation Loss Decreased(7.518422--->7.518405) 	 Saving The Model
Epoch 7 		 Training Loss: 0.6106977870831123 		 Validation Loss: 0.3132663667201996
Validation Lo

In [160]:
acc = 0
size = 0

with torch.no_grad():
    for (idx, emb, lab, sex, age) in test_data:

        embedding = emb
        age = age
        gender = sex

        outputs = net(embedding, age, gender)
        _, predictions = torch.max(outputs, 0)
        size += labels.size(0)
        acc += (predictions == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * acc // size} %')

Accuracy of the network on the 10000 test images: 50 %


/var/folders/nl/2sq00v7j1jxdw2y7s4_913tw0000gn/T/ipykernel_80289/2076255210.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  outputs = self.softmax(x)


In [173]:
with torch.no_grad():
    for (idx, emb, lab, sex, age) in test_data:

        embedding = emb
        age = age
        gender = sex

        outputs = net(embedding, age, gender)
        print(outputs)
        _, predictions = torch.max(outputs, 0)
        #print(predictions)

tensor([1.0000e+00, 2.8442e-06])
tensor([1.0000e+00, 2.8442e-06])
tensor([1.0000e+00, 2.8442e-06])
tensor([1.0000e+00, 2.8442e-06])
tensor([1.0000e+00, 2.8442e-06])
tensor([1.0000e+00, 2.8442e-06])
tensor([1.0000e+00, 2.8442e-06])
tensor([1.0000e+00, 2.8442e-06])
tensor([1.0000e+00, 2.8442e-06])
tensor([1.0000e+00, 2.8442e-06])
tensor([1.0000e+00, 2.8442e-06])
tensor([1.0000e+00, 2.8442e-06])
tensor([1.0000e+00, 2.8442e-06])
tensor([1.0000e+00, 2.8442e-06])
tensor([1.0000e+00, 2.8442e-06])
tensor([1.0000e+00, 2.8442e-06])
tensor([1.0000e+00, 2.8442e-06])
tensor([1.0000e+00, 2.8442e-06])
tensor([1.0000e+00, 2.8442e-06])
tensor([1.0000e+00, 2.8442e-06])
tensor([1.0000e+00, 2.8442e-06])
tensor([1.0000e+00, 2.8442e-06])
tensor([1.0000e+00, 2.8442e-06])
tensor([1.0000e+00, 2.8442e-06])
tensor([1.0000e+00, 2.8442e-06])


/var/folders/nl/2sq00v7j1jxdw2y7s4_913tw0000gn/T/ipykernel_80289/2076255210.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  outputs = self.softmax(x)


In [166]:
train_meta.shape

(195, 20)

In [169]:
labels = one_hot_encoding(get_df_info(df)[1], lab)

In [174]:
x

tensor([[-3.1478,  0.0852,  1.5616,  ...,  1.5419,  0.0097,  0.4409],
        [ 0.9053,  0.1356,  0.2012,  ..., -0.4302, -0.6142,  0.3462],
        [ 0.2356,  0.4305, -0.6886,  ...,  0.7071, -0.9203, -0.7698],
        ...,
        [ 0.5381,  1.0745, -0.5995,  ...,  0.1090,  0.3487,  1.3555],
        [-0.5861,  1.7550,  0.1783,  ..., -0.2784, -0.5937, -0.4141],
        [ 1.3645,  0.3365,  0.3683,  ...,  0.4707,  0.5664,  0.6351]])

In [183]:
arr = np.array(train_data.dataframe['embedding'])

In [187]:
x = torch.tensor([[1, 2, 3], [4, 5, 6]])

In [188]:
x

tensor([[1, 2, 3],
        [4, 5, 6]])

In [196]:
arr

array([tensor([[[ 0.1418,  0.0422, -0.0766,  ..., -0.1865, -0.1705, -0.0961],
         [ 0.0453,  0.2691, -0.2067,  ..., -0.3346,  0.1010, -0.1862],
         [-0.0052,  0.2907, -0.1201,  ..., -0.2853, -0.1145, -0.2356],
         ...,
         [ 0.2264,  0.0141, -0.0683,  ..., -0.0877, -0.3685, -0.0402],
         [-0.0080,  0.3318, -0.2290,  ..., -0.3211, -0.1260, -0.1756],
         [-0.0284, -0.0375, -0.0213,  ..., -0.0432, -0.0269, -0.0521]]]),
       tensor([[[ 0.2199,  0.1247, -0.0799,  ..., -0.0757, -0.1098, -0.1767],
         [ 0.0264,  0.2496, -0.2240,  ..., -0.2005,  0.1460, -0.2374],
         [ 0.0067,  0.3921, -0.0805,  ..., -0.1621, -0.1284, -0.3825],
         ...,
         [ 0.3356,  0.1211, -0.1553,  ...,  0.1328, -0.1887, -0.1962],
         [-0.0245,  0.3766, -0.1412,  ..., -0.2085, -0.1250, -0.1808],
         [-0.0217, -0.0245, -0.0236,  ..., -0.0403, -0.0163, -0.0557]]]),
       tensor([[[ 0.2559,  0.2160, -0.0711,  ...,  0.0591, -0.1684, -0.0971],
         [ 0.2036,  0.

In [204]:
x = torch.from_numpy(list(train_data.dataframe['embedding']))

AttributeError: 'list' object has no attribute 'values'

In [214]:
lli = list(train_data.dataframe['embedding'])

In [210]:
a = torch.stack(lli) 

In [212]:
a.shape

torch.Size([195, 1, 197, 768])

In [194]:
import torch
from torch.utils.data import DataLoader, TensorDataset


# Create the dataset with N_SAMPLES samples
N_SAMPLES, D_in, H, D_out = train_meta.shape[0], get_df_info(df)[0], 100, get_df_info(df)[1]


x = torch.randn(N_SAMPLES, D_in)
y = torch.randn(N_SAMPLES, D_out)

# Define the batch size and the number of epochs
BATCH_SIZE = 8
N_EPOCHS = 10

# Use torch.utils.data to create a DataLoader 
# that will take care of creating batches 
dataset = TensorDataset(x, y)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# Define model, loss and optimizer
#model = torch.nn.Sequential(
#    torch.nn.Linear(D_in, H),
#    torch.nn.ReLU(),
#    torch.nn.Linear(H, D_out),
#)

loss_fn = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)



# Get the dataset size for printing (it is equal to N_SAMPLES)
dataset_size = len(dataloader.dataset)

# Loop over epochs
for epoch in range(N_EPOCHS):
    print(f"Epoch {epoch + 1}\n-------------------------------")

    # Loop over batches in an epoch using DataLoader
    for id_batch, (x_batch, y_batch) in enumerate(dataloader):

        y_batch_pred = model(x_batch)

        loss = loss_fn(y_batch_pred, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Every 100 batches, print the loss for this batch
        # as well as the number of examples processed so far 
        if id_batch % 100 == 0:
            loss, current = loss.item(), (id_batch + 1)* len(x_batch)
            print(f"loss: {loss:>7f}  [{current:>5d}/{dataset_size:>5d}]")

Epoch 1
-------------------------------
loss: 10.385473  [    8/  195]
Epoch 2
-------------------------------
loss: 929.456787  [    8/  195]
Epoch 3
-------------------------------
loss: 256.186127  [    8/  195]
Epoch 4
-------------------------------
loss: 39.460579  [    8/  195]
Epoch 5
-------------------------------
loss: 7.019341  [    8/  195]
Epoch 6
-------------------------------
loss: 4.175135  [    8/  195]
Epoch 7
-------------------------------
loss: 5.890515  [    8/  195]
Epoch 8
-------------------------------
loss: 4.852470  [    8/  195]
Epoch 9
-------------------------------
loss: 2.521771  [    8/  195]
Epoch 10
-------------------------------
loss: 10.027683  [    8/  195]


In [121]:
net = FCNet(get_df_info(df,False)[0], get_df_info(df,False)[1])

In [122]:
#fair = False
import os
#from os import setxattr
import torch.optim as optim

epochs = 20
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(net.parameters(), lr=3e-4)

min_valid_loss = np.inf

for e in range(epochs):
    running_loss = 0.0  
    for (idx, emb, lab, sex, age) in train_data:
        optimizer.zero_grad()

        embedding = emb
        age = age
        gender = sex

        labels = one_hot_encoding(get_df_info(df,False)[1], lab)

        outputs = net(embedding, age, gender)
        # print(outputs, labels)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    # if i % 10 == 9:
    #   print(f'[{e + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
    #   running_loss = 0.0
  
    valid_loss = 0.0
    net.eval()
    for (idx, emb, lab, sex, age) in val_data:
        embedding = emb
        age = age
        gender = sex

        labels = one_hot_encoding(get_df_info(df)[1], lab)


        target = net(embedding, age, gender)
        loss = criterion(outputs, labels)
        valid_loss = loss.item() * len(val_data)

    print(f'Epoch {e+1} \t\t Training Loss: {running_loss / len(train_data)} \t\t Validation Loss: {valid_loss / len(val_data)}')

    if min_valid_loss > valid_loss:
        print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
        min_valid_loss = valid_loss
        # Saving State Dict
        torch.save(net.state_dict(), 'saved_model_unfair.pth')

/var/folders/nl/2sq00v7j1jxdw2y7s4_913tw0000gn/T/ipykernel_80289/2076255210.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  outputs = self.softmax(x)


Epoch 1 		 Training Loss: 0.6472305759405479 		 Validation Loss: 0.39044156670570374
Validation Loss Decreased(inf--->9.370598) 	 Saving The Model
Epoch 2 		 Training Loss: 0.6155618708867293 		 Validation Loss: 0.37154796719551086
Validation Loss Decreased(9.370598--->8.917151) 	 Saving The Model
Epoch 3 		 Training Loss: 0.6122638702392578 		 Validation Loss: 0.3620429039001465
Validation Loss Decreased(8.917151--->8.689030) 	 Saving The Model
Epoch 4 		 Training Loss: 0.6114324027147049 		 Validation Loss: 0.3561406135559082
Validation Loss Decreased(8.689030--->8.547375) 	 Saving The Model
Epoch 5 		 Training Loss: 0.6110292225311964 		 Validation Loss: 0.3520035147666931
Validation Loss Decreased(8.547375--->8.448084) 	 Saving The Model
Epoch 6 		 Training Loss: 0.6107946878824478 		 Validation Loss: 0.3488876223564148
Validation Loss Decreased(8.448084--->8.373303) 	 Saving The Model
Epoch 7 		 Training Loss: 0.610645465973096 		 Validation Loss: 0.3464280664920807
Validation Los

In [146]:
acc = 0
size = 0

with torch.no_grad():
    for (idx, emb, lab, sex, age) in test_data:

        embedding = emb
        age = age
        gender = sex

        outputs = net(embedding, age, gender)
        print(outputs)
        _, predictions = torch.max(outputs, 0)
        size += labels.size(0)
        acc += (predictions == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * acc // size} %')

tensor([0.9646, 0.0354])
tensor([0.9646, 0.0354])
tensor([0.9646, 0.0354])
tensor([0.9646, 0.0354])
tensor([0.9646, 0.0354])
tensor([0.9646, 0.0354])
tensor([0.9646, 0.0354])
tensor([0.9646, 0.0354])
tensor([0.9646, 0.0354])
tensor([0.9646, 0.0354])
tensor([0.9646, 0.0354])
tensor([0.9646, 0.0354])
tensor([0.9646, 0.0354])
tensor([0.9646, 0.0354])
tensor([0.9646, 0.0354])
tensor([0.9646, 0.0354])
tensor([0.9646, 0.0354])
tensor([0.9646, 0.0354])
tensor([0.9646, 0.0354])
tensor([0.9646, 0.0354])
tensor([0.9646, 0.0354])
tensor([0.9646, 0.0354])
tensor([0.9646, 0.0354])
tensor([0.9646, 0.0354])
tensor([0.9646, 0.0354])
Accuracy of the network on the 10000 test images: 50 %


/var/folders/nl/2sq00v7j1jxdw2y7s4_913tw0000gn/T/ipykernel_80289/2076255210.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  outputs = self.softmax(x)


In [133]:
_,predictions = torch.max(outputs, 0)

In [137]:
outputs

tensor([0.9646, 0.0354])

In [140]:
labels

tensor([1., 0.])

In [139]:
(predictions == labels).sum().item()

1

In [138]:
for (idx, emb, lab, sex, age) in test_data:
    print(idx)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
